In [1]:
import json
import time

from src.experiments.aux_code import *
from datasets.get_datasets import *
import time
from src.brkga.genetic import genetic as brkga
from src.brkga_variation.genetic import genetic as brkga_var
from src.s_genetic.genetic import *
from src.predicate import *

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
source='workedunder'
target='advisedby'

In [3]:
bk = json.loads(open('src/experiments/kb.txt').readline())


In [4]:
kb_source = bk['imdb']
kb_target = bk['uwcse']

In [5]:
pred_target = create_pred_target(kb_target)

In [6]:
imdb_dataset = datasets.load('imdb', kb_source, target=source, seed=441773, balanced=0) #facts, pos, neg
uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773, balanced=0) #facts, pos, neg

train_facts_source, train_pos_source, train_neg_source = get_train_division(imdb_dataset)
background_knowledge_src = boostsrl.modes(kb_source, [source], useStdLogicVariables=False, 
                                      maxTreeDepth=3, nodeSize=2, numOfClauses=8)
model_src = boostsrl.train(background_knowledge_src, train_pos_source, train_neg_source, train_facts_source, trees=10)

structured_src, new_src_struct = create_structured_trees(model_src)

In [7]:
uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773, balanced=0) #facts, pos, neg
train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 0)

0


# Crossover original

## Sem revisão

In [34]:
# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773, balanced=0) #facts, pos, neg
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 0)
# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773) #facts, pos, neg


start = time.time()
res = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30)
print("FIM DO GROOT GENETIC: ", time.time()-start)

KeyboardInterrupt: 

In [65]:
res 

(<src.s_genetic.population.Population at 0x108c92fa0>,
 [-1.0988574537039029,
  -1.0988574537039029,
  -1.13613036319567,
  -1.1556185269780042,
  -1.1556185269780042,
  -1.1593284677321285,
  -1.176123722941091,
  -1.176123722941091,
  -1.176123722941091,
  -1.176123722941091,
  -1.176123722941091,
  -1.176123722941091,
  -1.176123722941091,
  -1.176123722941091,
  -1.176123722941091],
 [{'m_auc_pr': 0.38296325,
   'm_auc_roc': 0.93727275,
   'm_cll': -0.57359175,
   'm_rec': 1.0,
   'm_pred': 0.291034625,
   'm_f1': 0.1507055,
   's_auc_pr': 0.08505931323310514,
   's_auc_roc': 0.00465756649845174,
   's_cll': 0.05384273926155225,
   's_rec': 0.0,
   's_prec': 0.20970956105822736,
   's_f1': 0.04266741949954321},
  {'m_auc_pr': 0.38296325,
   'm_auc_roc': 0.93727275,
   'm_cll': -0.57359175,
   'm_rec': 1.0,
   'm_pred': 0.291034625,
   'm_f1': 0.1507055,
   's_auc_pr': 0.08505931323310514,
   's_auc_roc': 0.00465756649845174,
   's_cll': 0.05384273926155225,
   's_rec': 0.0,
   's_p

In [14]:
def get_k_best_individuals(population, k):
    best_ind = []
    all_fitness = []
    for i in range(0, len(population)):
        all_fitness.append((population[i].fitness.values[0], i))
    
    sorted_ind = sorted(all_fitness, reverse=False, key=lambda tup: tup[0])
    for i in range(0, k):
        best_ind.append(population[sorted_ind[i][1]])
    return best_ind

In [54]:
best_cll = res[0].population[0].fitness.values[0]
best_auc = -res[0].population[0].fitness.values[1]
best = res[0].population[0]
for i in res[0].population:
    if i.fitness.values[0] > best_cll and i.fitness.values[1] > best_auc:
        best = i

In [68]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.937707, 'AUC PR': 0.29769, 'CLL': -0.744834, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x126ae0ca0>)
({'AUC ROC': 0.937707, 'AUC PR': 0.29769, 'CLL': -0.805205, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x126ae07f0>)
({'AUC ROC': 0.937707, 'AUC PR': 0.29769, 'CLL': -0.710564, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x126ae0f10>)
({'AUC ROC': 0.937707, 'AUC PR': 0.29769, 'CLL': -0.650665, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x126ae0460>)
INDIVIDUO  2
({'AUC ROC': 0.937707, 'AUC PR': 0.29769, 'CLL': -0.744834, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x126ae0bb0>)
({'AUC ROC': 0.937707, 'AUC PR': 0.29769, 'CLL': -0.805205, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <b

## Aleatório

In [119]:
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 0)
# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773) #facts, pos, neg


start = time.time()
res = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30)
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.63150625, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.04874941229068818, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  1
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.63150625, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.04874941229068818, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  2
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.67203875, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.06626196962200488, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  3
B

In [120]:
res

(<src.s_genetic.population.Population at 0x12773e2b0>,
 [-1.1076053369678323,
  -1.1076053369678323,
  -1.1312053395348367,
  -1.1312053395348367,
  -1.1312053395348367,
  -1.1312053395348367,
  -1.132072428976223,
  -1.132072428976223,
  -1.132072428976223,
  -1.132072428976223,
  -1.132072428976223,
  -1.132072428976223,
  -1.1342891893918081,
  -1.1342891893918081,
  -1.1342891893918081],
 [{'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.63150625,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.04874941229068818,
   's_rec': 0.0,
   's_prec': 0.21497987868344795,
   's_f1': 0.03626637711089984},
  {'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.63150625,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.04874941229068818,
   

In [121]:
def get_k_best_individuals(population, k):
    best_ind = []
    all_fitness = []
    for i in range(0, len(population)):
        all_fitness.append((population[i].fitness.values[0], i))
    
    sorted_ind = sorted(all_fitness, reverse=True, key=lambda tup: tup[0])
    for i in range(0, k):
        best_ind.append(population[sorted_ind[i][1]])
    return best_ind

In [122]:
final_results = {}
final_results[f'{source}->{target}'] = []
final_results[f'{source}->{target}'].append(res)
best = final_results[f'{source}->{target}'][0]
for i in final_results[f'{source}->{target}']:
    if i[1][-1] > best[1][-1]:
        best = i

In [125]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

# individuals = get_k_best_individuals(res[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.566956, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x1278afd30>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.760143, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x126ae0940>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.679182, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x1278af4c0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.616834, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x126ae0fa0>)
INDIVIDUO  2
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.71162, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x1278af4c0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.760143, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <bo

In [66]:
cll = [-0.371804,-0.371804,-0.371804]
pr = [0.942857,0.942857,0.942857]
roc = [0.942857,0.942857,0.942857]


import numpy as np
print("CLL: ", np.mean(cll), np.std(cll))
print("ROC: ", np.mean(roc), np.std(roc))
print("PR: ", np.mean(pr), np.std(pr))

CLL:  -0.371804 0.0
ROC:  0.942857 0.0
PR:  0.942857 0.0


In [24]:
final_results[f'brkga_{source}->{target}'] = []
final_results[f'inf_res_brkga:{source}->{target}'] = []
final_results[f'brkga_var_{source}->{target}'] = []
final_results[f'inf_res_brkga_var:{source}->{target}'] = []
save_results(final_results, source, target, 'imdb', 'uwcse')

In [25]:
final_results[f'groot_test:{source}->{target}']

[({'AUC ROC': 0.942857,
   'AUC PR': 0.897436,
   'CLL': -0.371804,
   'Precision': [0.897436, 0.5],
   'Recall': 1.0,
   'F1': 0.945946},
  <boostsrl.boostsrl.test at 0x13b4a68e0>),
 ({'AUC ROC': 0.942857,
   'AUC PR': 0.897436,
   'CLL': -0.371804,
   'Precision': [0.897436, 0.5],
   'Recall': 1.0,
   'F1': 0.945946},
  <boostsrl.boostsrl.test at 0x1077c8c40>),
 ({'AUC ROC': 0.942857,
   'AUC PR': 0.897436,
   'CLL': -0.371804,
   'Precision': [0.897436, 0.5],
   'Recall': 1.0,
   'F1': 0.945946},
  <boostsrl.boostsrl.test at 0x13b4a6d00>)]

## Apenas guiado

In [127]:
start = time.time()
res_g = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30)
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -1.092970248936728
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.6054705, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.0503632170959124, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  1
MELHOR RESULTADO:  -1.092970248936728
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.6054705, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.0503632170959124, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  2
MELHOR RESULTADO:  -1.1247142571042767
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.66105425, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 

In [142]:
res_g

(<src.s_genetic.population.Population at 0x125537580>,
 [-1.092970248936728,
  -1.092970248936728,
  -1.1247142571042767,
  -1.1278426691961794,
  -1.1588017600153024,
  -1.1588017600153024,
  -1.1588017600153024,
  -1.1588017600153024,
  -1.1588017600153024,
  -1.1588017600153024,
  -1.1588017600153024,
  -1.1588017600153024,
  -1.1588017600153024],
 [{'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.6054705,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.0503632170959124,
   's_rec': 0.0,
   's_prec': 0.21497987868344795,
   's_f1': 0.03626637711089984},
  {'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.6054705,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.0503632170959124,
   's_rec': 0.0,
   's_prec': 0.21497987868344795

In [134]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_g[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.7572, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x127433970>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.804689, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x127433a90>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.722383, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12843c6d0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.657902, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x127433a90>)
INDIVIDUO  2
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.7572, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12843c910>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.804689, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boost

In [67]:
cll = [-0.427185,-0.427185,-0.427185]
pr = [0.897436,0.897436,0.897436]
roc = [0.942857,0.942857,0.942857]


import numpy as np
print("CLL: ", np.mean(cll), np.std(cll))
print("ROC: ", np.mean(roc), np.std(roc))
print("PR: ", np.mean(pr), np.std(pr))

CLL:  -0.427185 0.0
ROC:  0.942857 0.0
PR:  0.897436 0.0


In [33]:
final_results[f'brkga_{source}->{target}'] = []
final_results[f'inf_res_brkga:{source}->{target}'] = []
final_results[f'brkga_var_{source}->{target}'] = []
final_results[f'inf_res_brkga_var:{source}->{target}'] = []
save_results(final_results, source, target, 'imdb', 'uwcse')

In [34]:
final_results[f'groot_test:{source}->{target}']

[({'AUC ROC': 0.942857,
   'AUC PR': 0.897436,
   'CLL': -0.427185,
   'Precision': [0.897436, 0.5],
   'Recall': 1.0,
   'F1': 0.945946},
  <boostsrl.boostsrl.test at 0x13b3b9a60>),
 ({'AUC ROC': 0.942857,
   'AUC PR': 0.897436,
   'CLL': -0.427185,
   'Precision': [0.897436, 0.5],
   'Recall': 1.0,
   'F1': 0.945946},
  <boostsrl.boostsrl.test at 0x13b3b9e20>),
 ({'AUC ROC': 0.942857,
   'AUC PR': 0.897436,
   'CLL': -0.427185,
   'Precision': [0.897436, 0.5],
   'Recall': 1.0,
   'F1': 0.945946},
  <boostsrl.boostsrl.test at 0x13b3b98e0>)]

## Mixed

In [135]:
start = time.time()
res = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30)
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -1.0718735101174777
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.566501, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.04964464285801641, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  1
MELHOR RESULTADO:  -1.0718735101174777
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.566501, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.04964464285801641, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  2
MELHOR RESULTADO:  -1.0718735101174777
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.566501, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 

In [141]:
res

(<src.s_genetic.population.Population at 0x12781b4c0>,
 [-1.0718735101174777,
  -1.0718735101174777,
  -1.0718735101174777,
  -1.0889420067196531,
  -1.0889420067196531,
  -1.1057602455554336,
  -1.1057602455554336,
  -1.1057602455554336,
  -1.1057602455554336,
  -1.1057602455554336,
  -1.1057602455554336,
  -1.1057602455554336,
  -1.1057602455554336,
  -1.1057602455554336],
 [{'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.566501,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.04964464285801641,
   's_rec': 0.0,
   's_prec': 0.21497987868344795,
   's_f1': 0.03626637711089984},
  {'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.566501,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.04964464285801641,
   's_rec': 0.0,
   's_p

In [140]:
final_results = {}
final_results[f'{source}->{target}'] = []
final_results[f'{source}->{target}'].append(res)
best = final_results[f'{source}->{target}'][0]
for i in final_results[f'{source}->{target}']:
    if i[1][-1] > best[1][-1]:
        best = i

In [143]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.657272, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x1269481c0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.690449, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x126948e50>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.632716, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x1269485b0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.577927, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x1269481c0>)
INDIVIDUO  2
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.657272, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x1269486a0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.690449, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <b

# Crossover modificado

## sem revisão

In [11]:
start = time.time()
res = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30)
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -1.1311193508761852
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.671894, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.05630018225991816, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  1
MELHOR RESULTADO:  -1.1311193508761852
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
entrei sem revisão
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.671894, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.05630018225991816, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  2
MELHOR R

In [15]:
res

(<src.s_genetic.population.Population at 0x12c8d91c0>,
 [-1.1311193508761852,
  -1.1311193508761852,
  -1.1311193508761852,
  -1.1311193508761852,
  -1.1348626097158745,
  -1.1465084992229941,
  -1.1465084992229941,
  -1.1465084992229941,
  -1.1465084992229941,
  -1.1465084992229941,
  -1.1670600723530153,
  -1.1670600723530153,
  -1.1670600723530153,
  -1.1670600723530153,
  -1.1670600723530153],
 [{'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.671894,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.05630018225991816,
   's_rec': 0.0,
   's_prec': 0.21497987868344795,
   's_f1': 0.03626637711089984},
  {'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.671894,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.05630018225991816,
 

In [16]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.769593, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12c4b5cd0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.817369, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12b4685e0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.734559, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12b468040>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.66966, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12b4687f0>)
INDIVIDUO  2
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.769593, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12b468820>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.817369, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <bo

## Aleatório

In [26]:
start = time.time()
res_r = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='random')
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -1.0702773072038678
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.563475, 'm_rec': 1.0, 'm_pred': 0.287192625, 'm_f1': 0.137791, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.04917586191923836, 's_rec': 0.0, 's_prec': 0.21330863588175322, 's_f1': 0.035325067374316496}
GENERATION:  1
MELHOR RESULTADO:  -1.0702773072038678
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.563475, 'm_rec': 1.0, 'm_pred': 0.287192625, 'm_f1': 0.137791, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.04917586191923836, 's_rec': 0.0, '

In [27]:
res_r

(<src.s_genetic.population.Population at 0x12bf1d790>,
 [-1.0702773072038678,
  -1.0702773072038678,
  -1.0702773072038678,
  -1.074336495445823,
  -1.074336495445823,
  -1.074336495445823,
  -1.074336495445823,
  -1.074336495445823,
  -1.074336495445823,
  -1.074336495445823,
  -1.074336495445823,
  -1.074336495445823],
 [{'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.563475,
   'm_rec': 1.0,
   'm_pred': 0.287192625,
   'm_f1': 0.137791,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.04917586191923836,
   's_rec': 0.0,
   's_prec': 0.21330863588175322,
   's_f1': 0.035325067374316496},
  {'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.563475,
   'm_rec': 1.0,
   'm_pred': 0.287192625,
   'm_f1': 0.137791,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.04917586191923836,
   's_rec': 0.0,
   's_prec': 0.21330863588175322,
   's_f1': 0.0353250673

In [28]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_r[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.604681, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12bb12730>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.638636, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12bc867f0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.579766, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12bb120a0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.525902, 'Precision': [0.044872, 0.5], 'Recall': 1.0, 'F1': 0.08589}, <boostsrl.boostsrl.test object at 0x12bc867f0>)
INDIVIDUO  2
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.604681, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12bb12790>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.638636, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <bo

## Guided

In [29]:
start = time.time()
res_g = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='guided')
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -1.0675340147162176
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.55824675, 'm_rec': 1.0, 'm_pred': 0.29189149999999997, 'm_f1': 0.15335325, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.031114475404343582, 's_rec': 0.0, 's_prec': 0.2090936646804489, 's_f1': 0.04759742488504499}
GENERATION:  1
MELHOR RESULTADO:  -1.0675340147162176
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.55824675, 'm_rec': 1.0, 'm_pred': 0.29189149999999997, 'm_f1': 0.15335325, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.031114475404343582, 's_rec': 

In [30]:
res_g

(<src.s_genetic.population.Population at 0x12b85d1f0>,
 [-1.0675340147162176,
  -1.0675340147162176,
  -1.0675340147162176,
  -1.0675340147162176,
  -1.0675340147162176,
  -1.0675340147162176,
  -1.0675340147162176,
  -1.0675340147162176,
  -1.0675340147162176],
 [{'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.55824675,
   'm_rec': 1.0,
   'm_pred': 0.29189149999999997,
   'm_f1': 0.15335325,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.031114475404343582,
   's_rec': 0.0,
   's_prec': 0.2090936646804489,
   's_f1': 0.04759742488504499},
  {'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.55824675,
   'm_rec': 1.0,
   'm_pred': 0.29189149999999997,
   'm_f1': 0.15335325,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.031114475404343582,
   's_rec': 0.0,
   's_prec': 0.2090936646804489,
   's_f1': 0.04759742488504499},
  {'m_auc_pr': 0.17426725,


In [31]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_g[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.57913, 'Precision': [0.044872, 0.5], 'Recall': 1.0, 'F1': 0.08589}, <boostsrl.boostsrl.test object at 0x12c52b6a0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.599783, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12bb12730>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.563719, 'Precision': [0.044872, 0.5], 'Recall': 1.0, 'F1': 0.08589}, <boostsrl.boostsrl.test object at 0x12c40dc70>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.534539, 'Precision': [0.044872, 0.5], 'Recall': 1.0, 'F1': 0.08589}, <boostsrl.boostsrl.test object at 0x12bb12700>)
INDIVIDUO  2
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.57913, 'Precision': [0.044872, 0.5], 'Recall': 1.0, 'F1': 0.08589}, <boostsrl.boostsrl.test object at 0x12c4b5910>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.599783, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsr

## Mixed

In [23]:
start = time.time()
res_m = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='mixed')
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -1.1094026259325895
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.6346532500000001, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.054658340691860575, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  1
MELHOR RESULTADO:  -1.1094026259325895
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.6346532500000001, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.054658340691860575, 's_rec': 0.0, 's

In [24]:
res_m

(<src.s_genetic.population.Population at 0x12b5085e0>,
 [-1.1094026259325895,
  -1.1094026259325895,
  -1.1094026259325895,
  -1.1094026259325895,
  -1.1094026259325895,
  -1.1094026259325895,
  -1.1094026259325895,
  -1.1113716797775453,
  -1.1113716797775453,
  -1.1140241633580599,
  -1.1141969763906763,
  -1.1150832325206157,
  -1.1150832325206157,
  -1.1150832325206157,
  -1.1150832325206157],
 [{'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.6346532500000001,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.054658340691860575,
   's_rec': 0.0,
   's_prec': 0.21497987868344795,
   's_f1': 0.03626637711089984},
  {'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.6346532500000001,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0

In [25]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_m[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.6682, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12b4681c0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.714954, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12bc867f0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.655864, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12bc86c40>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.604685, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12b4681c0>)
INDIVIDUO  2
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.6682, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12bc86c40>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.714954, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boost

## Teste TreeBoostler

In [ ]:
tr_train_pos = []
tr_train_neg = []
tr_train_facts = []

for i in range(0, len(train_pos)):
    tr_train_pos.extend(train_pos[i])
    tr_train_neg.extend(train_neg[i])
    tr_train_facts.extend(train_facts[i])

train = [tr_train_pos, tr_train_neg, tr_train_facts]
aa = test_tree_b(source, target, kb_source, kb_target, structured_src, train, test)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-51-6d08d6867bf0>", line 11, in <module>
    aa = test_tree_b(source, target, kb_source, kb_target, structured_src, train, test)
  File "/Users/leticia.figueiredo/Documents/faculdade/groot/src/experiments/aux_code.py", line 200, in test_tree_b
    background = boostsrl.modes(kb_target, [to_predicate], useStdLogicVariables=False, maxTreeDepth=3, nodeSize=2, numOfClauses=8)
  File "/usr/local/lib/python3.9/site-packages/boostsrl/boostsrl.py", line 208, in __init__
    write_to_file(background_knowledge, 'boostsrl/background.txt')
  File "/usr/local/lib/python3.9/site-packages/boostsrl/boostsrl.py", line 96, in write_to_file
    with open(path, 'w') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'boostsrl/background.txt'

During handling of the above exception, 

In [62]:
aa

 {'AUC ROC': 0.971429,
  'AUC PR': 0.964905,
  'CLL': -0.246793,
  'Precision': [0.911765, 0.5],
  'Recall': 0.885714,
  'F1': 0.898551,
  'Learning time': 29.751,
  'Inference time': 1.994},
 [['advisedby(A, B)',
   {'': 'student(A), professor(B)',
    'true': 'tempadvisedby(C, B), ta(D, C, E)',
    'true,true': 'ta(D, A, F)',
    'true,false': 'publication(G, B)',
    'true,false,true': 'publication(G, H), student(H)'},
   {'true,true,true': [0.894, 1, 4],
    'true,true,false': [1.46e-07, 0, 26],
    'true,false,true,true': [1.664, 3, 36],
    'true,false,true,false': [1.69, 4, 10],
    'true,false,false': [1.0, 2, 2],
    'false': [4.71e-08, 68, 0]}],
  ['advisedby(A, B)',
   {'': 'professor(B), student(A)',
    'true': 'publication(C, B), publication(C, A)',
    'true,true': 'tempadvisedby(D, B), ta(E, A, F)',
    'true,true,true': 'ta(G, H, F), ta(G, D, I)',
    'true,false': 'publication(J, A)',
    'true,false,true': 'publication(J, K), professor(K)',
    'true,false,false': 'p

In [64]:
background_knowledge_src = boostsrl.modes(kb_target, [target], useStdLogicVariables=False, 
                                      maxTreeDepth=3, nodeSize=2, numOfClauses=8)
sys.path.insert(0, '../TreeBoostler')

from revision import revision

def print_function(message):
        global experiment_title
        global nbr
        experiment_title = f'Twitter->Yeast2: {source}->{target}'
        nbr = 1
        if not os.path.exists('experiments/' + experiment_title):
            os.makedirs('experiments/' + experiment_title)
        with open('experiments/' + experiment_title + '/' + str(nbr) + '_' + experiment_title + '.txt', 'a') as f:
            print(message, file=f)
            print(message)

model = revision.learn_test_model(background_knowledge_src, boostsrl, target, tr_train_pos, tr_train_neg, tr_train_facts, test_pos, test_neg, test_facts, trees=10, print_function=print_function)

WILL Produced-Tree #1
% FOR advisedby(A, B):
%   if ( student(A), professor(B) )
%   then if ( tempadvisedby(C, B), ta(D, C, E) )
%   | then if ( ta(D, A, F) )
%   | | then return 0.6581489350995122;  // std dev = 0,894, 5,000 (wgt'ed) examples reached here.  /* #neg=1 #pos=4 */
%   | | else return 0.8581489350995118;  // std dev = 1,46e-07, 26,000 (wgt'ed) examples reached here.  /* #pos=26 */
%   | else if ( publication(G, B) )
%   | | then if ( publication(G, H), student(H) )
%   | | | then return 0.7812258581764345;  // std dev = 1,664, 39,000 (wgt'ed) examples reached here.  /* #neg=3 #pos=36 */
%   | | | else return 0.5724346493852266;  // std dev = 1,690, 14,000 (wgt'ed) examples reached here.  /* #neg=4 #pos=10 */
%   | | else return 0.3581489350995122;  // std dev = 1,000, 4,000 (wgt'ed) examples reached here.  /* #neg=2 #pos=2 */
%   else return -0.1418510649004877;  // std dev = 4,71e-08, 68,000 (wgt'ed) examples reached here.  /* #neg=68 */
WILL Produced-Tree #2
% FOR advis

Results
   AUC ROC   = 0.973878
   AUC PR    = 0.966421
   CLL	      = -0.239925
   Precision = 0.914286 at threshold = 0.5
   Recall    = 0.914286
   F1        = 0.914286


Total learning time: 41.323 seconds
Total inference time: 2.536 seconds
AUC ROC: 0.973878




In [65]:
model

 {'AUC ROC': 0.973878,
  'AUC PR': 0.966421,
  'CLL': -0.239925,
  'Precision': [0.914286, 0.5],
  'Recall': 0.914286,
  'F1': 0.914286,
  'Learning time': 41.323,
  'Inference time': 2.536},
 [['advisedby(A, B)',
   {'': 'student(A), professor(B)',
    'true': 'tempadvisedby(C, B), ta(D, C, E)',
    'true,true': 'ta(D, A, F)',
    'true,false': 'publication(G, B)',
    'true,false,true': 'publication(G, H), student(H)'},
   {'true,true,true': [0.894, 1, 4],
    'true,true,false': [1.46e-07, 0, 26],
    'true,false,true,true': [1.664, 3, 36],
    'true,false,true,false': [1.69, 4, 10],
    'true,false,false': [1.0, 2, 2],
    'false': [4.71e-08, 68, 0]}],
  ['advisedby(A, B)',
   {'': 'student(A), professor(B)',
    'true': 'tempadvisedby(C, B), ta(D, C, E)',
    'true,true': 'ta(D, A, F)',
    'true,false': 'publication(G, B)',
    'true,false,true': 'publication(H, A), tempadvisedby(I, B)'},
   {'true,true,true': [0.894, 1, 4],
    'true,true,false': [0.0, 0, 26],
    'true,false,tru